# Рекомендация тарифов

В распоряжении данные о поведении клиентов, которые уже перешли на тарифы *"Смарт"* и *"Ультра"* из [Определение перспективного тарифа для телеком-компании](https://github.com/Jilopchick/Data-Science/tree/main/Projects-from-courses/Determining-a-promising-tariff-for-a-telecom-company). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится, так как она уже была выполнена в предыдущем проекте.

Необходимо построить модель с максимально большим значением `accuracy`. Чтобы считать проект выполненным, нужно довести долю правильных ответов по крайней мере до 0.75.

## Изучение данных из файла

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
import numpy as np

In [2]:
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('users_behavior.csv')

In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df.duplicated().sum()

0

Данные в порядке, дубликатов нет. В предобработке не нужнаются.

Подробнее опишем значения каждого атрибута.

Таблица `df`(информация о поведении пользователей за месяц):
- `calls` - количество звонков;
- `minutes` - суммарная длительность звонков в минутах;
- `messages` - количество сообщений;
- `mb_used` - израсходованный интернет-трафик в Мб;
- `is_ultra` - каким тарифом пользовался в течении месяца (1 - *"Ультра"*, 0 - *"Смарт"*).

## Разбить данные

Разобрьём наши данные на обучающую, валидационную и тестовую выборки. Развер обучающей выборки *50%*, валидационной - *25%*, тестовой - *25%*.

In [7]:
features_train, features_valid, target_train, target_valid = train_test_split(df.drop(['is_ultra'], axis=1),
                                                                              df['is_ultra'], 
                                                                              test_size=0.5, 
                                                                              random_state=1337)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5, 
                                                                            random_state=1337)

## Исследование разных моделей

### Дерево решений

In [8]:
best_accuracy = 0
best_depth = 0
for depth in range(1, 101):
    model = DecisionTreeClassifier(max_depth=depth, random_state=1337)
    model.fit(features_train, target_train)
    accuracy = accuracy_score(target_valid, model.predict(features_valid))
    best_accuracy, best_depth = (accuracy, depth) if accuracy > best_accuracy else (best_accuracy, best_depth)

In [9]:
print('best accuracy =', best_accuracy, '\nbest depth =', best_depth)

best accuracy = 0.8119551681195517 
best depth = 7


Таким образом, наилучшее значение `accuracy` достигается при грубине дерева равной 7, а само значение `accuracy` примерно равно 0.81.

### Случайный лес

In [10]:
best_accuracy = 0
best_depth = 0
best_estimators = 0
for est in range(2, 101):
    for depth in range(1, 101):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=1337)
        model.fit(features_train, target_train)
        accuracy = accuracy_score(target_valid, model.predict(features_valid))
        best_accuracy, best_estimators, best_depth = (accuracy, est, depth) if accuracy > best_accuracy else (best_accuracy, best_estimators, best_depth)

In [11]:
print('best accuracy =', best_accuracy, '\nbest estimators =', best_estimators, '\nbest depth =', best_depth)

best accuracy = 0.8293897882938979 
best estimators = 34 
best depth = 9


Наилучшее значение `accuracy` достигается при количестве деревьев равному 34, глубине дерева - 9, а само значение `accuracy` примерно равно 0.83.

### Логистическая регрессия

In [12]:
solvers = ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']
penaltys = ['l1', 'l2', 'none']
best_solver = ''
best_penalty = ''
best_accuracy = 0

for solv in solvers:
    for pen in penaltys:
        try: 
            model = LogisticRegression(solver=solv, max_iter=1000, penalty=pen, random_state=1337)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            best_accuracy, best_solver, best_penalty = (accuracy, solv, pen) if accuracy > best_accuracy else (best_accuracy, best_solver, best_penalty)
        except: 
            continue

In [13]:
print('best accuracy =', best_accuracy, '\nbest solver =', best_solver, '\nbest penalty =', best_penalty)

best accuracy = 0.7621419676214197 
best solver = lbfgs 
best penalty = l2


Наилучшее значение `accuracy` достигается при алгоритме оптимизации *lbfgs* и штрафе *l2*, значение `accuracy` получается примерно равным 0.76.

Здесь вы использовали `model.score()`, что равноценно `accuracy_score()`. Преимущество такого метода в том, что не надо предварительно использовать `.predict()`.

**Проверив модели на валидационной выборке были следаны следующие выводы:**
- самой точной моделью является **Случайный лес**, опираясь на оценку `accuracy`;
- самыми быстрыми моделями являются **Дерево решений** и **Логистическая регрессия**;
- остановимся на модели **Случайный лес**, хоть она и медленне остальных двух, зато имеет самую высокую точность.

## Проверка качества модели на тестовой выборке

In [14]:
model = RandomForestClassifier(max_depth=9, n_estimators=34, random_state=1337)
model.fit(features_train, target_train)
print('Accuracy тестовой выборки =', model.score(features_test, target_test))

Accuracy тестовой выборки = 0.7935323383084577


`Accuracy` упало на 0.04. Однако результат **Случайного леса** на тестовой выборкевсё ещё удовлитворительный.

Дополнительно можно изучить и остальные модели с "наилучшими" гиперпараметрами.

**Дерево решений**

In [15]:
model = DecisionTreeClassifier(max_depth=7, random_state=1337)
model.fit(features_train, target_train)
print('Accuracy тестовой выборки =', model.score(features_test, target_test))

Accuracy тестовой выборки = 0.7848258706467661


**Логистическая регрессия**

In [16]:
model = LogisticRegression(solver='lbfgs', max_iter=1000, penalty='l2', random_state=1337)
model.fit(features_train, target_train)
print('Accuracy тестовой выборки =', model.score(features_test, target_test))

Accuracy тестовой выборки = 0.7313432835820896


Показатели `accuracy` моделей **Дерево решений** и **Логистическая регрессия** тоже упали.

**Вывод:**
- `accuracy` модели на тестовой выборке приемлемо упала.

## Проверка модели на адекватность

Для проверки на адекватность нашей модели напишем функцию, предсказывающую значение признака is_ultra самым простым способом - 50/50. Сравним, лучше ли наша модель.

In [17]:
random_prediction = np.random.randint(low=0, high=2, size=target_test.shape[0])
print('accuracy =', accuracy_score(target_test, random_prediction))

accuracy = 0.48880597014925375


Как видно, случайные ответыошибаются примерно в *50%* случаев. Модель **Случайного леса** ошибается примерно в *20%* случаев.

**Выводы:**
- модель **Случайного леса** точнее подкидывания монетки;
- модель прошла проверку на адекватность/вменяемость.